In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df, df2db
from astro_utils import mjd_to_datetime
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_mjds, integrate_mjds_df
from ra_dec import calc_topos
# from ztf_data_load import db_engine, load_ztf_classes, load_ztf_objects, load_ztf_detections, ztf_detection_add_dir

In [3]:
# Distinct ZTF detection times
dt = sp2df('ZTF.GetDetectionTimes')

In [4]:
# dt[0:5]

In [5]:
# Filter down dt for testing
# dt = dt.loc[0:1000]

In [6]:
# DataFrame for output-- augmented set of detection times
df = dt.copy()

In [7]:
# Array of observations times as MJDs
mjds = dt['MJD'].values

In [8]:
# Populate the CalendarDateTime field
df['CalendarDateTime'] = np.array([mjd_to_datetime(mjd) for mjd in mjds])

In [9]:
# All available data sources as a DataFrame
ds = sp2df('KS.GetDataSources')

In [10]:
# Populate DataSourceID and ObservatoryID fields
df['DataSourceID'] = ds.DataSourceID[ds.DataSourceCD=='ZTF'].values[0]
df['ObservatoryID'] = ds.ObservatoryID[ds.DataSourceCD=='ZTF'].values[0]

In [11]:
# Calculate Earth position
sim_epoch = make_sim_planets(epoch=mjds[0])

In [12]:
# Integrate the planets
# df_p = integrate_mjds_df(sim_epoch=sim_epoch, mjds=mjds, save_elements=False, progbar=True)

In [13]:
# df_p[0:5]

In [14]:
# Integrate the planets saving outputs at these observation times
body_ids, body_names, q, v, elts = integrate_mjds(sim_epoch=sim_epoch, mjds=mjds, save_elements=False, progbar=True)

  0%|          | 0/216702 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [15]:
# Earth position at these observation times
earth_idx = np.argmax(body_names=='Earth')
q_earth = q[:,earth_idx,:]
v_earth = v[:,earth_idx,:]

In [16]:
# Calculate topos adjustment
dq_topos, dv_topos = calc_topos(obstime_mjd=mjds, site_name='Palomar')

In [17]:
dq_topos.shape

(216702, 3)

In [18]:
# The position and velocity of the observatory
q_obs = q_earth + dq_topos.value # in AU
v_obs = v_earth + dv_topos.value # in AU / day

In [19]:
# Position of the Sun
sun_idx = np.argmax(body_names=='Sun')
q_sun = q[:,sun_idx,:]

In [20]:
# Save observatory position to augmented detection time DataFrame
df[['qObs_x', 'qObs_y', 'qObs_z']] = q_obs
df[['vObs_x', 'vObs_y', 'vObs_z']] = v_obs
df[['qSun_x', 'qSun_y', 'qSun_z']] = q_sun

In [21]:
df

,DetectionTimeID,MJD,CalendarDateTime,DataSourceID,ObservatoryID,qObs_x,qObs_y,qObs_z,vObs_x,vObs_y,vObs_z,qSun_x,qSun_y,qSun_z
0,1,58270.167662,2018-06-01 04:01:25.996783,1,1,-0.342267,-0.947311,-0.000032,0.015945,-0.006084,0.000088,0.000841,0.006808,-0.000097
1,2,58270.168125,2018-06-01 04:02:05.999991,1,1,-0.342260,-0.947314,-0.000032,0.015946,-0.006083,0.000088,0.000841,0.006808,-0.000097
2,3,58270.169965,2018-06-01 04:04:45.001903,1,1,-0.342231,-0.947325,-0.000032,0.015949,-0.006082,0.000088,0.000841,0.006808,-0.000097
3,4,58270.170417,2018-06-01 04:05:24.002897,1,1,-0.342223,-0.947328,-0.000032,0.015949,-0.006082,0.000088,0.000841,0.006808,-0.000097
4,5,58270.171343,2018-06-01 04:06:44.000648,1,1,-0.342209,-0.947333,-0.000032,0.015951,-0.006082,0.000088,0.000841,0.006808,-0.000097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216697,216698,59295.526690,2021-03-22 12:38:25.998720,1,1,-1.003215,-0.023336,0.000160,0.000431,-0.017329,0.000027,-0.007178,0.005510,0.000123
216698,216699,59295.527153,2021-03-22 12:39:06.001920,1,1,-1.003215,-0.023344,0.000161,0.000432,-0.017329,0.000027,-0.007178,0.005510,0.000123
216699,216700,59295.527627,2021-03-22 12:39:46.998720,1,1,-1.003215,-0.023352,0.000161,0.000432,-0.017328,0.000027,-0.007178,0.005510,0.000123
216700,216701,59295.528090,2021-03-22 12:40:27.001920,1,1,-1.003214,-0.023360,0.000161,0.000432,-0.017327,0.000026,-0.007178,0.005510,0.000123


In [22]:
bound_

NameError: name 'bound_' is not defined